# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=4194f89a822c5bf4998e975c44343e3f7d3d03e677aea5761a76b6775d0820c9
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=Z9GXg2sa3Lxz0DwTMqLU0M5s0dAnTz&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Uh6rgPUT6qA7WubXhQgfqTihzRCPf35A81NPFH4Gg5Q&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJjh7fvjqRMYXBAnHK5oOemR31LTFwXiEICSY6GgGkwF9Yw3QsuysLyNQe11Qm_3Dw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

# Tratamento

In [ ]:
# Importa a biblioteca pandas-gbq, necessária para interagir com o BigQuery.
import pandas_gbq

# Define o ID do projeto do Google Cloud onde a consulta BigQuery será executada.
project_id = "repositoriodedadosgpsp"

# Define a consulta SQL a ser executada no BigQuery.
# A consulta agrega a quantidade de vínculos de trabalho por ano, UF, esfera e poder.
query = '''SELECT
  ano,
  sigla_uf,
  esfera,
  poderes,
  quantidade_vinculos
FROM (
  SELECT
    ano,
    sigla_uf,
    CASE
      WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
      WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
      WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
      ELSE 'Outros'
    END AS esfera,
    CASE
      WHEN natureza_juridica IN ('1015','1023', '1031','1104','1139','1112', '1147','1236','1120', '1155','1180','1341', '1163','1252','1260','1279','1287','1295','1309','1317','1325','1333','1171','1180', '1244', '1341') THEN 'Executivo'
      WHEN natureza_juridica IN ('1040','1058', '1066') THEN 'Legislativo'
      WHEN natureza_juridica IN ('1074','1082') THEN 'Judiciário'
      ELSE 'Outros'
    END AS poderes,
    COUNT(*) AS quantidade_vinculos
  FROM `repositoriodedadosgpsp.Datalake.RAIS_2024_ALL_AJUSTADO`
  GROUP BY ano, sigla_uf, esfera, natureza_juridica
) subquery
WHERE esfera != 'Outros' and poderes != 'Outros'
ORDER BY esfera, ano, sigla_uf, poderes;


---116-3 - Órgão Público Autônomo Federal - executivo
--125-2 - Fundação Pública de Direito Privado Federal
--126-0 - Fundação Pública de Direito Privado Estadual ou do Distrito Federal
--127-9 - Fundação Pública de Direito Privado Municipal
--128-7 - Fundo Público da Administração Indireta Federal
--129-5 - Fundo Público da Administração Indireta Estadual ou do Distrito Federal
--130-9 - Fundo Público da Administração Indireta Municipal
--131-7 - Fundo Público da Administração Direta Federal
--132-5 - Fundo Público da Administração Direta Estadual ou do Distrito Federal
--133-3 - Fundo Público da Administração Direta Municipal
--117-1 - Órgão Público Autônomo Estadual ou do Distrito Federal - executivo
--118-0 - Órgão Público Autônomo Municipal - executivo
--124-4 - Município
--134-1 - União '''

# Executa a consulta no BigQuery e armazena os resultados em um DataFrame do pandas.
df = pandas_gbq.read_gbq(query, project_id=project_id)

# Mostra as primeiras linhas do DataFrame para uma rápida visualização dos dados.
print(df.head())

# Upload

In [ ]:
# Importa a biblioteca do BigQuery, necessária para interagir com a API.
from google.cloud import bigquery

# Define o esquema da tabela de destino no BigQuery, especificando nome, tipo e descrição de cada coluna.
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência'),
          bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description= 'Número total de vinculos observados'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação'),
          bigquery.SchemaField('esfera', 'STRING', description= 'Nível da esfera do governo'),
          bigquery.SchemaField('poderes', 'STRING', description= 'Poder abrangente ao nível de esfera')
          ]

## Subindo para datalake
# Inicializa o cliente do BigQuery, especificando o projeto a ser utilizado.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Cria uma referência ao dataset (conjunto de dados) de destino no BigQuery.
dataset_ref = client.dataset('perfil_remuneracao')

# Cria uma referência à tabela de destino dentro do dataset especificado.
table_ref = dataset_ref.table('RAIS_evolucao_esfera_poder_v1')

# Configura o trabalho de carregamento (load job), aplicando o esquema definido anteriormente.
job_config = bigquery.LoadJobConfig(schema=schema)

# Inicia o trabalho de upload do DataFrame para a tabela no BigQuery, usando a configuração definida.
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Aguarda a conclusão do trabalho de upload e verifica se ocorreram erros.
job.result()